In [1]:
import numpy as np
np.set_printoptions(precision=4)
import matplotlib.pyplot as plt
import PIL

% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
print(np.__version__) # 1.13.3
print(PIL.__version__) # 4.3.0

1.13.3
4.3.0


In [3]:
from modules.create_images import open_image
from modules.create_images import save_derived_image
from modules.create_images import downscale_image
from modules.create_images import add_to_background
from modules.create_images import make_random_size

In [4]:
foreground_image_files = ['dog.png', 'tree.png', 'woman.png']
background_image_files = ['city.jpg', 'street.jpg', 'landscape.jpg']

foreground_images = [open_image(f) for f in foreground_image_files]
background_images = [None] + [downscale_image(open_image(b), 128, 128)
                     for b in background_image_files]


In [5]:
save_creations = True

for i in range(10):
    l, t, w, h = make_random_size()
    added = add_to_background(
        foreground_images[np.random.choice(len(foreground_images))],
        l, t, w, h,
        background_images[np.random.choice(len(background_images))],
    )
    added.show()
    if save_creations:
        save_derived_image(added)
